# Introduction to Parallel Programming

This notebook shows how to write, compile, and run parallel program

## Shared Memory Parallelism

Here’s a simple "Hello World" program using OpenMP in C. OpenMP is an API that supports multi-platform shared memory multiprocessing programming in C, C++, and Fortran.

Example OpenMP Program: `openmp_hello_world.c`

```
#include <stdio.h>
#include <omp.h>
#include <unistd.h>  // for gethostname
#include <limits.h>  // for HOST_NAME_MAX

int main() {
    // Variable to store the hostname
    char hostname[HOST_NAME_MAX];
    
    // Get the hostname of the machine
    gethostname(hostname, HOST_NAME_MAX);

    // Set number of threads
    omp_set_num_threads(4);
    
    // Parallel region starts here
    #pragma omp parallel
    {
        // Each thread will execute this code block
        int thread_id = omp_get_thread_num();
        int num_threads = omp_get_num_threads();
        
        printf("Hello World from thread %d out of %d threads running on %s\n", thread_id, num_threads, hostname);
    }

    return 0;
}
```

Explanation:
- `#pragma omp parallel`: This directive tells the compiler to parallelize the following block of code. Each thread will execute the code inside this block.
- `omp_get_thread_num()`: This function returns the ID of the current thread, which ranges from 0 to num_threads-1.
- `omp_get_num_threads()`: This function returns the total number of threads in the parallel region.

**1. Compile the Program:**

To compile an OpenMP program, you need to use the `-fopenmp` flag with gcc:

```$ gcc -fopenmp -o hello_world/openmp_hello_world.exe hello_world/openmp_hello_world.c```

In [2]:
!gcc -fopenmp -o hello_world/openmp_hello_world.exe hello_world/openmp_hello_world.c

**2. Run the Program:**

Run the compiled program:

```$ ./hello_world/openmp_hello_world.exe```

In [3]:
!./hello_world/openmp_hello_world.exe

Hello World from thread 1 out of 4 threads running on grid1
Hello World from thread 3 out of 4 threads running on grid1
Hello World from thread 2 out of 4 threads running on grid1
Hello World from thread 0 out of 4 threads running on grid1


## Distributed Memory Parallelism

Here’s a simple "Hello World" program using MPI in C. MPI is a library that supports multi-node distributed memory parallelism programming in C, C++, and Fortran.

Example MPI Program: `mpi_hello_world.c`

```
#include <mpi.h>
#include <stdio.h>
#include <unistd.h>

int main(int argc, char** argv) {
    MPI_Init(NULL, NULL);
    
    int world_size;
    MPI_Comm_size(MPI_COMM_WORLD, &world_size);
    
    int world_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &world_rank);
    
    char hostname[256];
    gethostname(hostname, sizeof(hostname));

    printf("Hello world from rank %d out of %d processors running on %s\n", world_rank, world_size, hostname);
    
    MPI_Finalize();
    return 0;
}

```

**1. Compile the Program:**

To compile an MPI program, you need to use the mpicc:

```$ mpicc -o mpi_hello_world mpi_hello_world.c```

**2. Run the Program:**

To run an MPI program, you need to use the mpirun. But you need to define nodes and number of slot (processors) that can be used in MPI program.

Example `hostfile`

```
grid1 slots=2
grid2 slots=2
```

Thus, you can execute the MPI program by defining hostfile and the number of processor (that should be less than total number of slots in hostfile.

```$ mpirun -np 4 -hostfile hostfile mpi_hello_world```

Unfortunately, the xeus-cling kernel is designed for interactive C++ programming and doesn't have built-in support for MPI compilation. The MPI program have to be compiled and run outside of the xeus-cling kernel. This means you won't get interactive output as you would with regular C++ code in xeus-cling.